# Election Results Sonification

With the results of the last french presidential election (1st round).

In [ ]:
import ipytone
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
import ipyleaflet
import xyzservices.providers as xyz
from shapely.geometry import Point

## Read and prepare data

In [ ]:
candidates = {
    "Arthaud": 1,
    "Roussel": 2,
    "Macron": 3,
    "Lassalle": 4,
    "LePen": 5,
    "Zemmour": 6,
    "Mélenchon": 7,
    "Hidalgo": 8,
    "Jadot": 9,
    "Pécresse": 10,
    "Poutou": 11,
    "Dupont-Aignan": 12,
}

Election results data.

source: https://www.data.gouv.fr/fr/datasets/election-presidentielle-des-10-et-24-avril-2022-resultats-definitifs-du-1er-tour/#resources

In [ ]:
df = pd.read_excel("data/resultats-par-niveau-subcom-t1-france-entiere.xlsx")

In [ ]:
n_candidates = len(candidates)
vote_col1 = 24
vote_col_gap = 7

vote_cols = range(vote_col1, vote_col1 + n_candidates * vote_col_gap, vote_col_gap)

In [ ]:
df_votes = df.iloc[:, list(vote_cols)]

df_votes_perc = df_votes.div(df_votes.sum(axis=1), axis=0)

insee = df["Code du département"].astype(str) + df["Code de la commune"].astype(str)
df_votes_perc = df_votes_perc.set_index(insee)

Localities admin borders

source: https://www.data.gouv.fr/en/datasets/decoupage-administratif-communal-francais-issu-d-openstreetmap/

In [ ]:
gdf_communes = (
    gpd.read_file("data/communes-20220101-shp/communes-20220101.shp")
    .set_index("insee")
)

Merge the two datasets and compute the centroid of the localities.

In [ ]:
gdata = gdf_communes.join(df_votes_perc)

In [ ]:
# French local territories have limited extent so
# using geographic CRS should be ok
gdata.geometry = gdf_communes.centroid

France admin borders (for display).

source: https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/

In [ ]:
nuts = gpd.read_file("data/NUTS_RG_01M_2021_4326.shp/NUTS_RG_01M_2021_4326.shp")

france = nuts.query("LEVL_CODE == 0 and NUTS_NAME == 'France'")

## Sound setup

Panning values for each candidate given their political side, from far left (-1) to far right (+1). This is "somewhat" arbitrary.

In [ ]:
candidates_pan = {
    "Arthaud": -0.9,
    "Roussel": -0.7,
    "Macron": 0.3,
    "Lassalle": 0,
    "LePen": 0.9,
    "Zemmour": 0.9,
    "Mélenchon": -0.7,
    "Hidalgo": -0.4,
    "Jadot": -0.3,
    "Pécresse": 0.5,
    "Poutou": -0.9,
    "Dupont-Aignan": 0.9,
}

Prepare players with speech samples for each candidate.

In [ ]:
base_url = "http://localhost:8888/files/Projects/ipytone/examples/data/election_candidates_wav/"
suffix = ".wav?_xsrf=2%7C1ced6df3%7C0514fa79e3ccfbcffefe3f864a0d4032%7C1654092692"

urls = {name: base_url + name + suffix for name in candidates}
players = {}
pans = {}
gains = {}

for name in candidates:
    url = base_url + name + suffix
    player = ipytone.Player(url, loop=True, fade_in=0.1, fade_out=0.1)
    gain = ipytone.Gain(gain=0)
    pan = ipytone.Panner(pan=candidates_pan[name])
    player.chain(gain, pan, ipytone.destination)
    
    players[name] = player
    pans[name] = pan
    gains[name] = gain
  

Start all players and adjust volumes

In [ ]:
for p in players.values():
    p.start()
    p.volume.value = 5

## Map, widgets and interactions

Map widget

In [ ]:
m = ipyleaflet.Map(
    zoom=6,
    center=[46.90, 5.10],
    basemap=xyz.CartoDB.Positron,
    layout=widgets.Layout(height="600px")
)

france_outline = ipyleaflet.GeoData(
    geo_dataframe=france,
    style={
        'color': 'black',
        'fillColor': '#ffffff',
        'opacity': 0.2,
        'weight': 2,
        'fillOpacity': 0.2
    },
    name='Countries',
)

m.add_layer(france_outline)
m.add_control(ipyleaflet.LayersControl())

Gain value progress widgets

In [ ]:
commune_label = widgets.Label()

levels = []

for name in candidates:
    label = widgets.Label(value=name, layout=widgets.Layout(width="100px"))
    level = widgets.FloatProgress(min=0, max=1, value=0)
    widgets.jsdlink((gains[name].gain, "value"), (level, "value"))
    levels.append(widgets.HBox([label, level]))

levels_box = widgets.VBox(
    [commune_label] + levels,
    layout=widgets.Layout(padding="20px"),
)

levels_control = ipyleaflet.WidgetControl(
    widget=levels_box, position='topright'
)

m.add_control(levels_control)

Map interaction (cursor position will adjust player gains).

In [ ]:
col_offset = 3

def update_gains(**kwargs):
    if kwargs.get('type') == 'mousemove':
        lat, lon = kwargs.get('coordinates')
        point = Point(lon, lat)
        
        
        if not france.contains(point).squeeze():
            for gain in gains.values():
                gain.gain.value = 0.0
            commune_label.value = "Locality: none"
        else:
            _, idx = gdata.sindex.nearest(point)
        
            slocality = gdata.iloc[idx.item()]
            
            commune_label.value = f"Locality: {slocality.nom}"
            
            for name, idx in candidates.items():
                col = idx + col_offset
                new_value = slocality[col]
                gains[name].gain.value = new_value


m.on_interaction(update_gains)

In [ ]:
m

## Clean-up

In [ ]:
for p in players.values():
    p.stop()

In [ ]:
for p in players.values():
    p.dispose()
    
for g in gains.values():
    g.dispose()
    
for p in pans.values():
    p.dispose()